In [4]:
%%pyspark
df = spark.read.load('abfss://raw@clauserawstore.dfs.core.windows.net/NYCYellowtaxi/Yellow cab/yellow_tripdata_2019-03.csv', format='csv'
## If header exists uncomment line below
, header=True
)

df.createOrReplaceTempView('yellow_tripdata')

StatementMeta(Clausedemo, 2, 4, Finished, Available)

In [5]:
df.createOrReplaceTempView('yellow_tripdata')

StatementMeta(Clausedemo, 2, 5, Finished, Available)

In [6]:
from pyspark.sql import *
from pyspark.sql.types import *

StatementMeta(Clausedemo, 2, 6, Finished, Available)

In [7]:
%%sql
-- Create Delta Lake table, define schema and location
CREATE TABLE IF NOT EXISTS default.Yellow_Tripdata (
    VendorID INT, 
    tpep_pickup_datetime TIMESTAMP, 
    tpep_dropoff_datetime TIMESTAMP, 
    passenger_count INT, 
    trip_distance DECIMAL, 
    RateCodeID INT, 
    store_and_fwd_flag STRING, 
    PULocationID INT, 
    DOLocationID INT, 
    payment_type INT, 
    fare_amount DECIMAL, 
    extra DECIMAL, 
    mta_tax DECIMAL, 
    tip_amount DECIMAL, 
    tolls_amount DECIMAL, 
    improvement_surcharge DECIMAL, 
    total_amount DECIMAL, 
    congestion_surcharge STRING
    ) USING DELTA
LOCATION '/Delta_Demo/YellowTripdata/' 

StatementMeta(Clausedemo, 2, 7, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [8]:
%%sql

INSERT INTO default.Yellow_Tripdata SELECT * FROM yellow_tripdata

StatementMeta(Clausedemo, 2, 8, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [9]:
%%sql
SELECT * 
FROM default.Yellow_Tripdata
ORDER BY PULocationID

StatementMeta(Clausedemo, 2, 9, Finished, Available)

<Spark SQL result set with 1000 rows and 18 fields>

In [11]:
%%sql
UPDATE default.yellow_tripdata SET VendorID = 2 WHERE PULocationID = 1 AND tpep_dropoff_datetime = "2019-01-26T15:05:39Z"

StatementMeta(Clausedemo, 2, 11, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [12]:
%%sql
-- Retrieve the version/change history of the Delta table
DESCRIBE HISTORY  default.Yellow_Tripdata

StatementMeta(Clausedemo, 2, 12, Finished, Available)

<Spark SQL result set with 6 rows and 15 fields>

In [14]:
# Load a previous version of the Yellow_Tripdata table into a dataframe
df = spark.read.format("delta").option("versionAsOf", 1).load("/Delta_Demo/YellowTripdata/")
df.show()

StatementMeta(Clausedemo, 2, 14, Finished, Available)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RateCodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1| 2019-01-01 00:46:40|  2019-01-01 00:53:20|              1|            2|         1|                 N|         151|         239|           1|          7|    1|      1|         2|           0|                    0